In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as mpl
import scipy as sp
import pandas as pd
from mayavi import mlab

In [ ]:
sys.path.append("./bin/")
import ZernikeFunc as ZF
import SurfaceFunc as SF
import MyUtils as MU  # my personal collection of functions

# Initial parameters 

In [ ]:
# Parameters definition
Npixel = 25    # the plane edge in pixels
Rs = 6         # the radius of the sphere that includes the patch
Dpp = 0.5      # the distance between points of the same patch (needed to remove islands)
center = 5000  # point of the surface used as center of the patch (default index value)
threshold = 5  # value to enstablish variance is high (in Angstrom units)

In [ ]:
### To load a new surface:
surf_name_a = "./data/4bs2_RRM2.dms"
surf_a_ = pd.read_csv(surf_name_a)  
l_a = len(surf_a_["x"])
print("Npoints", l_a)
surf_a = np.zeros((l_a, 6))
surf_a[:,:] = surf_a_[["x", "y", "z", "Nx", "Ny", "Nz"]]

In [ ]:
### To inizialize the Surface class:
surf_a_obj = SF.Surface(surf_a[:,:], patch_num = 0, r0 = Rs, theta_max = 45)

## 3D graphs of the surface

In [ ]:
# To plot entire surface in 3D:
res1, c = SF.ConcatenateFigPlots([surf_a_obj.surface[:,:3]])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
# plotting surface + patch
res1, c = SF.ConcatenateFigPlots([surf_a_obj.surface[:,:3],patch[:,:3]])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
# plotting patch only
res1, c = SF.ConcatenateFigPlots([patch[:,:3]])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
# plotting patch + colored center of mass
cm = np.mean(patch[:,:3], axis=0)
res1, c = SF.ConcatenateFigPlots([patch[:,:3],np.row_stack([cm,cm])])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
# plotting patch + rotated patch
tmp1 = patch[:,:3] - np.mean(patch[:,:3], axis=0)
tmp2 = rot_patch[:,:3]
res1, c = SF.ConcatenateFigPlots([tmp1,tmp2])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
# plotting rotated patch with normal vectors
SF.Plot3DPointsAndVectors(rot_patch[:,0], rot_patch[:,1], rot_patch[:,2],rot_patch_nv[:,0], rot_patch_nv[:,1], rot_patch_nv[:,2])

# Percentage of non-functionality with the first method

In [ ]:
## To isolate one patch of the surface:
patch, mask = surf_a_obj.BuildPatch(point_pos=center, Dmin=Dpp)

In [ ]:
## To rotate a patch:    
rot_patch, rot_patch_nv = surf_a_obj.PatchReorientNew(patch, +1)
# +1 : vettori normali verso l'alto (default)
# -1 : vettori normali verso il basso

In [ ]:
## To project the patch on the xy plane...
z = surf_a_obj.FindOrigin(rot_patch, 0)  # mettere 1 al posto di 0 per plottare patch ruotata + cono
# per ottenere le medie e le varianze della patch sul piano xy
plane, plane_var, _, _, _ = MU.CreatePlane_Weights("", patch=rot_patch, z_c=z, Np=Npixel)

In [ ]:
# To calculate percentage of non-funzionality within unitary disk respect to a certain threshold
# The RuntimeWarning is provided in the libraries
_, perc = MU.PercHigherVariance_Weights("var", Npixel, surf_a_obj, center, Dpp, threshold)
print("Percentage of high variances on unit disk is {}".format(perc))

In [ ]:
MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "")

# To save figure
#MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "Point_{}_Weights".format(center))

The patch with this center is a true function (the percentage of non-functionality is zero) because there is no variance higher than the threshold value.  
The greater the number of pixels with variance above the threshold, the higher the non-functionality fraction of the chosen patch.

## Percentage of non-functionality for each patch

In [ ]:
limit = l_a
step = 1
points_list = np.arange(0,limit,step)
perc = np.zeros((len(points_list)))

for i in range(len(points_list)) :
    _, _, perc[i] = MU.PercHigherVariance_Weights(Npixel, Rs, surf_a_obj, points_list[i], Dpp, threshold)
    
print("Number of patches =",len(perc))

In [ ]:
with open("all_perc.txt", "w") as file0 :
    for i in range(len(points_list)) :
        file0.write("{}\t{}\n".format(points_list[i],perc[i]))

In [ ]:
points_list = np.loadtxt("./risultati/all_perc.txt", usecols=0, unpack=True)
perc = np.loadtxt("./risultati/all_perc.txt", usecols=1, unpack=True)

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(8,4), facecolor="white", dpi=200)

ax.set_xlim(0, len(perc))
ax.set_ylim(0, np.amax(perc)+0.01)

ax.set_title("Threshold = {}, Points = {}, Pixels = {}, Dpp = {}, Rs = {}".format(threshold,len(points_list),Npixel,Dpp,Rs), fontsize="8")
ax.set_xlabel("Surface point", fontsize="8")
ax.set_ylabel("Percentage", fontsize="8")

ax.tick_params(axis="both", width ="0.30", color="black", labelsize="6")
ax.locator_params(axis="x", nbins=21)
ax.locator_params(axis="y", nbins=21)
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.30)
    ax.spines[side].set_color("black")

ax.plot(points_list, perc, "o", markersize="0.4", rasterized=True)

fig.tight_layout()
#mpl.savefig("all_perc.pdf")

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(8,4), facecolor="white", dpi=200)

ax.set_title("Threshold = {} $\AA$, Points = {}, Pixels = {}, Dpp = {}, Rs = {}".format(threshold,len(points_list),Npixel,Dpp,Rs), fontsize="8")
ax.set_xlabel("Percentage", fontsize="8")
ax.set_ylabel("Number of values", fontsize="8")

ax.tick_params(axis="both", width ="0.30", color="black", labelsize="6")
ax.locator_params(axis="x", nbins=20)
ax.locator_params(axis="y", nbins=20)
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.30)
    ax.spines[side].set_color("black")

ax.hist(perc, bins=int(np.sqrt(len(perc))), histtype="step", rasterized=True)

fig.tight_layout()
mpl.savefig("hist_01.pdf")

Therefore, for the surface under examination, the best points are those corresponding to the minimums of the percentage, while the worst ones are those corresponding to the maximums.  
The absolute maximum and a local maximum are identified below.

In [ ]:
ipmax, _, _, _ = MU.zone_extremes(perc, 0, len(perc))
plane, plane_var, _ = MU.PercHigherVariance_Weights("", Npixel, surf_a_obj, ipmax, Dpp, threshold)
MU.PlotMeanVariancePatch(ipmax, Dpp, Rs, perc[ipmax], threshold, plane, plane_var, [], "")

# To save figure
#MU.PlotMeanVariancePatch(ipmax, Dpp, Rs, perc[ipmax], threshold, plane, plane_var, [], "Point_{}_Weights".format(ipmax))

In [ ]:
ipmax, _, _, _ = MU.zone_extremes(perc, 8000, 12000)
plane, plane_var, _ = MU.PercHigherVariance_Weights("", Npixel, surf_a_obj, ipmax, Dpp, threshold)
MU.PlotMeanVariancePatch(ipmax, Dpp, Rs, perc[ipmax], threshold, plane, plane_var, [], "")

# To save figure
#MU.PlotMeanVariancePatch(ipmax, Dpp, Rs, perc[ipmax], threshold, plane, plane_var, [], "Point_{}_Weights".format(ipmax))

# Percentage of non-functionality with the second method
An attempt to reduce the percentage of non-functionality. 

In [ ]:
center = 5000
plane, plane_var, perc = MU.PercHigherVariance_Projections("", Npixel, surf_a_obj, center, Dpp, threshold)
MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "")

# To save figure
#MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "Point_{}_Projections".format(center))

In [ ]:
center = 19841
plane, plane_var, perc = MU.PercHigherVariance_Projections("", Npixel, surf_a_obj, center, Dpp, threshold)
MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "")

# To save figure
#MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "Point_{}_Projections".format(center))

In [ ]:
center = 8227
plane, plane_var, perc = MU.PercHigherVariance_Projections("", Npixel, surf_a_obj, center, Dpp, threshold)
MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "")

# To save figure
#MU.PlotMeanVariancePatch(center, Dpp, Rs, perc, threshold, plane, plane_var, [], "Point_{}_Projections".format(center))

## Percentage of non-functionality for each patch

In [ ]:
limit = l_a
step = 1
points_list = np.arange(0,limit,step)
perc = np.zeros((len(points_list)))

for i in range(len(points_list)) :
    _, _, perc[i] = MU.PercHigherVariance_Projections(Npixel, Rs, surf_a_obj, points_list[i], Dpp, threshold)
    
print("Number of patches =",len(perc))

In [ ]:
with open("all_perc_projections.txt", "w") as file0b :
    for i in range(len(points_list)) :
        file0b.write("{}\t{}\n".format(points_list[i],perc[i]))

In [ ]:
# prendo indici delle patch e relative percentuali dal file
points_list = np.loadtxt("./risultati/all_perc_projections.txt", usecols=0, unpack=True)
perc = np.loadtxt("./risultati/all_perc_projections.txt", usecols=1, unpack=True)

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(8,4), facecolor="white", dpi=200)

ax.set_xlim(0, len(perc))
ax.set_ylim(0, np.amax(perc)+0.01)

ax.set_title("Threshold = {}, Points = {}, Pixels = {}, Dpp = {}, Rs = {}".format(threshold,len(points_list),Npixel,Dpp,Rs), fontsize="8")
ax.set_xlabel("Surface point", fontsize="8")
ax.set_ylabel("Percentage", fontsize="8")

ax.tick_params(axis="both", width ="0.30", color="black", labelsize="6")
ax.locator_params(axis="x", nbins=21)
ax.locator_params(axis="y", nbins=21)
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.30)
    ax.spines[side].set_color("black")

ax.plot(points_list, perc, "o", markersize="0.4", rasterized=True)

fig.tight_layout()
#mpl.savefig("all_perc_proj.pdf")

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(8,4), facecolor="white", dpi=200)

ax.set_title("Threshold = {} $\AA$, Points = {}, Pixels = {}, Dpp = {}, Rs = {}".format(threshold,len(points_list),Npixel,Dpp,Rs), fontsize="8")
ax.set_xlabel("Percentage", fontsize="8")
ax.set_ylabel("Number of values", fontsize="8")

ax.tick_params(axis="both", width ="0.30", color="black", labelsize="6")
ax.locator_params(axis="x", nbins=20)
ax.locator_params(axis="y", nbins=20)
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.30)
    ax.spines[side].set_color("black")

ax.hist(perc, bins=int(np.sqrt(len(perc))), histtype="step", rasterized=True)

fig.tight_layout()
#mpl.savefig("hist_02.pdf")